In [ ]:
import argparse

import numpy as np
from PIL import Image
import cv2

import torch
import torchvision

from data_loader import get_loader
from models_4 import VqaModel, SANModel
import warnings 

In [ ]:
def load_str_list(fname):
    with open(fname) as f:
        lines = f.readlines()
    lines = [l.strip() for l in lines]
    return lines

In [ ]:
qst_vocab = load_str_list("datasets/vocab_questions.txt")
ans_vocab = load_str_list("datasets/vocab_answers.txt")

In [ ]:
word2idx_dict = {w:n_w for n_w, w in enumerate(qst_vocab)}
unk2idx = word2idx_dict['<unk>'] if '<unk>' in word2idx_dict else None
vocab_size = len(qst_vocab)

In [ ]:
def word2idx(w):
    if w in word2idx_dict:
        return word2idx_dict[w]
    elif unk2idx is not None:
         return unk2idx
 
    else:
        raise ValueError('word %s not in dictionary (while dictionary does not contain <unk>)' % w)

In [ ]:
def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([224, 224], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

In [ ]:
def visualizeAttention(model, img, layer):
    
    pi = model.attn_features[layer].squeeze()
    print(pi.size())
    
    pi = pi.view(14,14)
    attn = m(pi)
    
    image = image.squeeze(0)
    img = torch.numpy(img)
    attn  = torch.numpy(attn)
    
#     print(image.shape, attn.shape)
    ## Visualization yet to be completed

In [ ]:
input_dir='./datasets
log_dir='./logs'
model_dir='./models'

max_qst_length=30
max_num_ans=10
embed_size=1024
word_embed_size=300

num_layers=2
hidden_size=512

num_epochs=30
batch_size=256

In [ ]:
data_loader = get_loader(
    input_dir=args.input_dir,
    
    input_vqa_train='train.npy',
    input_vqa_valid='valid.npy',
    
    max_qst_length=args.max_qst_length,
    max_num_ans=args.max_num_ans,
    
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    
    subset=None
)

In [ ]:
qst_vocab_size = data_loader['train'].dataset.qst_vocab.vocab_size
ans_vocab_size = data_loader['train'].dataset.ans_vocab.vocab_size
ans_unk_idx = data_loader['train'].dataset.ans_vocab.unk2idx

In [ ]:
model = VqaModel(
    embed_size=args.embed_size,
    qst_vocab_size=qst_vocab_size,
    ans_vocab_size=ans_vocab_size,
    word_embed_size=args.word_embed_size,
    num_layers=args.num_layers,
    # hidden_size=args.hidden_size).to(device)
    hidden_size=args.hidden_size,
    stack_size=args.stack_size
)

In [ ]:
image_path = ''

In [ ]:
image = cv2.imread(args.image_path)
image = cv2.resize(image, dsize=(224, 224), interpolation = cv2.INTER_AREA)

image = torch.from_numpy(image).float()
image = image.to(device)

image = image.unsqueeze(dim=0)
image = image.view(1, 3, 224, 224)

In [ ]:
max_qst_length=30

question = args.question
q_list = list(question.split(" "))
#     print(q_list)

In [ ]:
idx = 'valid'

qst2idc = np.array([word2idx('<pad>')] * max_qst_length)  # padded with '<pad>' in 'ans_vocab'
qst2idc[:len(q_list)] = [word2idx(w) for w in q_list]

question = qst2idc
question = torch.from_numpy(question).long()

question = question.to(device)
question = question.unsqueeze(dim=0)

In [ ]:
model = torch.load(args.saved_model)
model = model.to(device)

#torch.cuda.empty_cache()
model.eval()

output = model(image, question)

#     Visualization yet to be implemented
    if model.__class__.__name__ == "VQAModel":
        print(model.attn_features[0].size())
         visualizeAttention(model, image, layer=0)

predicts = torch.softmax(output, 1)
probs, indices = torch.topk(predicts, k=5, dim=1)

probs = probs.squeeze()
indices = indices.squeeze()
print("predicted - probabilty")
for i in range(5):
#         print(probs.size(), indices.size())
#         print(ans_vocab[indices[1].item()],probs[1].item())
    print("'{}' - {:.4f}".format(ans_vocab[indices[i].item()], probs[i].item()))